<a href="https://colab.research.google.com/github/Pantopelagienne/TP_Note_XAI/blob/main/TP_XAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  Comprendre son modèle de Machine Learning grâce à l'Explicabilité

[Google Form](https://docs.google.com/forms/d/e/1FAIpQLSeE30hbSmbQO8vjiT6UJdk9hAR6xOTpDks71E6BBre1dea0AA/viewform?usp=sf_link)

# Imports

In [ ]:
# Installer les packages requis
!pip install pycaret explainerdashboard pycaret[analysis] --upgrade jupyter-dash shap lime

In [ ]:
# Importer les librairies nécessaires
import pandas as pd
from pycaret.classification import *
from explainerdashboard import ClassifierExplainer, ExplainerDashboard
import shap
import lime.lime_tabular
from sklearn.preprocessing import LabelEncoder
from sklearn.datasets import load_iris
from sklearn.cluster import KMeans
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input, decode_predictions
from tensorflow.keras.preprocessing import image
from lime import lime_image
from skimage.segmentation import mark_boundaries

# 1- Utilisation d'un Explainer Dashboard

**Objectif** : Dans cette première partie, vous apprendrez à créer  et analyser un tableau de bord interactif avec la librairie ExplainerDashboard qui vous aidera à visualiser et comprendre les performances et les prédictions du modèle.

Google Form : Page 2 Partie SHAP, Page 2 Partie Contrefactuelles

## Chargement des données

Caractéristiques du dataset :

*   Age
*   Sexe
* Job : Niveau de compétence et type de résidence (0 - unskilled and non-resident, 1 - unskilled and resident, 2 - skilled, 3 - highly skilled)
* Housing : Type de logement (own, rent, or free)
* Saving accounts : Niveau des comptes d'épargne (little, moderate, quite rich, rich)
* Checking account : Montant des comptes courant
* Credit amount : Montant du crédit
* Duration : Durée du crédit en mois
* Purpose : Objet du crédit (car, furniture/equipment, radio/TV, domestic appliances, repairs, education, business, vacation/others)

In [ ]:
# Charger le dataset
dataset = pd.read_csv("/path_to_your_file", sep=",")
dataset.drop(columns=dataset.columns[0], axis=1, inplace=True)

In [ ]:
# Définir la variable cible
target = "Risk"

## Entraînement du modèle

In [ ]:
# Setup l'environnement PyCaret
exp1 = setup(dataset, target=target, session_id=123)

In [ ]:
# Comparer les modèles
best_model = compare_models()

In [ ]:
# Entraîner le meilleur modèle
model = create_model(best_model)

In [ ]:
# Finaliser le modèle
final_model = finalize_model(model)

## Création de l'Explainer Dashboard

In [ ]:
# Remplir les valeurs manquantes pour les variables catégorielles
categorical_columns = ["Sex", "Housing", "Saving accounts", "Checking account", "Purpose"]
dataset[categorical_columns] = dataset[categorical_columns].fillna("missing")

In [ ]:
X = dataset.drop(columns=['Risk'])
y = get_config("y_transformed")

In [ ]:
# Créer un explainer pour le modèle
explainer = ClassifierExplainer(final_model, X, y, labels=['bad','good'])

In [ ]:
# Créer le dashboard
dashboard = ExplainerDashboard(explainer,
  importances=True,
  model_summary=False,
  contributions=True,
  whatif=True,
  shap_dependence=False,
  shap_interaction=False,
  decision_trees=False,
  title="Credit Risk Model Explainer",
  description="This dashboard provides insights into the credit risk model using SHAP.")

In [ ]:
# Lancez le dashboard
dashboard.run(port=8050)

# 2- Comparaison de SHAP et LIME

**Objectif** : Cette partie vous permettra de comparer deux méthodes d'explicabilité, SHAP et LIME, en les appliquant à un dataset de classification bien connu, le dataset Iris.

Google Form : Page 3

In [ ]:
# Charger le dataset Iris
iris = load_iris()
X = pd.DataFrame(iris.data, columns=iris.feature_names)
y = iris.target

Mettez en place un nouvel environnement PyCaret, entraînez un random forest (pour pouvoir calculer TreeShap par la suite)

In [ ]:
#

Initialisez un explainer LIME et un explainer SHAP, sélectionnez une instance du dataset en particulier puis comparez les résultats (à l'aide de bar plots par exemple)

In [ ]:
#

# 3- Expliquer les prédictions d'un modèle avec LIME (images)

**Objectif** : Dans cette section, vous allez utiliser LIME pour expliquer les prédictions d'un modèle de classification d'images pré-entraîné, VGG16. Vous apprendrez à identifier et interpréter les régions de l'image qui influencent le plus les prédictions du modèle, vous permettant ainsi de mieux comprendre comment et pourquoi le modèle prend certaines décisions.

Google Form : Page 4

In [ ]:
# Charger le modèle pré-entraîné VGG16
model = VGG16(weights='imagenet')

In [ ]:
# Fonction pour charger et prétraiter l'image
def load_image(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = preprocess_input(img)
    return img

Choisissez une image à étiqueter

In [ ]:
# Chemin de l'image à étiquetter
img_path = "/path_to_your_file"

In [ ]:
# Charger et prétraiter l'image
img = load_image(img_path)

In [ ]:
# Prédire l'étiquette de l'image
preds = model.predict(img)
print('Predicted:', decode_predictions(preds, top=3)[0])

In [ ]:
# Fonction de prédiction pour LIME
def predict(images):
    images = preprocess_input(images)
    return model.predict(images)

In [ ]:
# Initialiser l'explainer de LIME
explainer = lime_image.LimeImageExplainer()

In [ ]:
# Expliquer la prédiction de l'image
explanation = explainer.explain_instance(img[0].astype('double'),
                                         predict,
                                         top_labels=40,
                                         hide_color=0,
                                         num_samples=1000)

In [ ]:
# Obtenir le superpixel et le masque pour l'explication
temp, mask = explanation.get_image_and_mask(explanation.top_labels[0],
                                            posit ive_only=True,
                                            num_features=5,
                                            hide_rest=False)

In [ ]:
# Afficher les superpixels importants
plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.imshow(mark_boundaries(temp / 255.0, mask))
plt.title("Superpixels importants")

# 4- Clusters d'explications

**Objectif** : Dans cette section, vous allez explorer comment les valeurs SHAP peuvent être utilisées pour créer des clusters d'explications.

Références : [How to make the most of local explanations: effective
clustering based on influences](https://hal.science/hal-04189455/document)

Utilisez un algorithme de clustering (ex : K-means) sur les explications SHAP ou LIME d'un modèle. Vous pouvez réutiliser celles calculées dans la partie 2. Libre à vous de choisir un nouveau dataset et d'entraîner un nouveau modèle (sur un problème de classification)

In [ ]:
#

Comparez ensuite les résultats obtenus avec les classifications des points d'origine

In [ ]:
#